In [21]:
import os
import pandas as pd

In [22]:
MASTER_FILE = 'data/master.data'
COLUMNS = [
	'id',
	'dataset',
	'heuristic_type',
	'heuristic',
	'is_baseline',
	'population',
	'burn_in',
	'replay',
	'reselection',
	'reanalysis',
	'normalisation',
	'credit',
	'discounted_rewards',
	'run',
	'step',
	'wall_time',
	'tag',
	'value'
]
TYPES = {
	'id': 'category',
	'dataset': 'category',
	'heuristic_type': 'category',
	'heuristic': 'category',
	'is_baseline': 'boolean',
	'population': 'Int32',
	'burn_in': 'Int32',
	'replay': 'Int32',
	'reselection': 'Int32',
	'reanalysis': 'Int32',
	'normalisation': 'boolean',
	'credit': 'category',
	'discounted_rewards': 'boolean',
	'run': 'Int32',
	'step': 'Int32',
	'wall_time': 'float32',
	'tag': 'category',
	'value': 'float32'
}

In [23]:
master_filepath = os.path.join(os.getcwd(), '..', MASTER_FILE);
raw = pd.read_csv(master_filepath, names=COLUMNS, dtype=TYPES, skiprows=1)
raw.head()

,id,dataset,heuristic_type,heuristic,is_baseline,population,burn_in,replay,reselection,reanalysis,normalisation,credit,discounted_rewards,run,step,wall_time,tag,value
0,ckxtgk52q0000qdp0gfc8fyc2,car,bhh,bhh_gd_only,False,5,0,10,1,1,True,symmetric,True,9,0,1.640773e+09,train_loss,1.622783
1,ckxtgk5310001qdp07kxe366d,car,bhh,bhh_gd_only,False,5,0,10,1,1,True,symmetric,True,9,0,1.640773e+09,train_accuracy,0.345152
2,ckxtgk5350002qdp01ebj0to7,car,bhh,bhh_gd_only,False,5,0,10,1,1,True,symmetric,True,9,0,1.640773e+09,test_loss,1.614782
3,ckxtgk53a0003qdp0ahnvwt64,car,bhh,bhh_gd_only,False,5,0,10,1,1,True,symmetric,True,9,0,1.640773e+09,test_accuracy,0.335260
4,ckxtgk53e0004qdp0hztp24te,car,bhh,bhh_gd_only,False,5,0,10,1,1,True,symmetric,True,9,1,1.640773e+09,train_loss,0.776363


In [24]:
raw.describe()

,is_baseline,population,burn_in,replay,reselection,reanalysis,normalisation,discounted_rewards,run,step,wall_time,value
count,389700,389700.000000,389700.000000,389700.000000,389700.000000,389700.000000,389700,389700,486300.000000,486300.000000,4.863000e+05,486300.000000
unique,2,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
top,False,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN
freq,370380,NaN,NaN,NaN,NaN,NaN,389700,389700,NaN,NaN,NaN,NaN
mean,NaN,6.989325,8.172132,20.292635,8.310105,8.981832,NaN,NaN,5.489858,9.997462,1.640778e+09,3.844857
std,NaN,11.601294,22.845426,46.083889,22.436637,23.367377,NaN,NaN,2.872044,6.055214,4.880449e+04,372.313782
min,NaN,5.000000,0.000000,10.000000,1.000000,1.000000,NaN,NaN,1.000000,0.000000,1.640724e+09,0.000000
25%,NaN,5.000000,0.000000,10.000000,1.000000,1.000000,NaN,NaN,3.000000,5.000000,1.640735e+09,0.157428
50%,NaN,5.000000,0.000000,10.000000,1.000000,1.000000,NaN,NaN,5.000000,10.000000,1.640764e+09,0.319199
75%,NaN,5.000000,0.000000,10.000000,1.000000,1.000000,NaN,NaN,8.000000,15.000000,1.640812e+09,0.893344


In [25]:
raw.dtypes

id                    category
dataset               category
heuristic_type        category
heuristic             category
is_baseline            boolean
population               Int32
burn_in                  Int32
replay                   Int32
reselection              Int32
reanalysis               Int32
normalisation          boolean
credit                category
discounted_rewards     boolean
run                      Int32
step                     Int32
wall_time              float32
tag                   category
value                  float32
dtype: object

In [26]:
INDICES = [
	'dataset',
	'heuristic',
	'population',
	'burn_in',
	'replay',
	'reselection',
	'reanalysis',
	'normalisation',
	'credit',
	'discounted_rewards',
	# 'run',
	'step',
]
COLUMNS='tag'
VALUES='value'
AGGFUNC='mean'
flattened = pd.pivot_table(
	raw,
	index=INDICES,
	columns=COLUMNS,
	values=VALUES,
	aggfunc='mean'
)

In [ ]:
flattened.to_csv('test.csv')